Following codes are based on following document, opened by the auther.

https://giggleliu.github.io/TwoQubit-VQE.html

This notebook is aimed to understand quantum tensor network and research the performance and building block technic.

# Solving TFI model with only 2 qubits - the 幺 simulation

Reference: Variational Quantum Eigensolver with Fewer Qubits

Jin-Guo Liu, Yi-Hong Zhang, Yuan Wan, Lei Wang

In [1]:
using Yao
using Statistics: mean
using LinearAlgebra
using Plots

### Build a quantum circuit inspired by MPS

The goal of this section is to build the MPS-inspired sampler as our ansatz

In [41]:
rotor(noleading::Bool=false, notrailing::Bool=false) = noleading ? (notrailing ? Rx(0) : chain(Rx(0), Rz(0))) : (notrailing ? chain(Rz(0), Rx(0)) : chain(Rz(0), Rx(0), Rz(0)))

# TT: Rx(0), TF: Rx(0)Rz(0), FT: Rz(0), FF: Rz(0)Rx(0)Rz(0)
# 0にはパラメータが後で入る

"""
    twoqubit_circuit(nlayer::Int, nrepeat::Int)

Construct the above ansatz, `nrepeat` is the number of measure operations,
`nlayer` is the length of each block.
"""
function twoqubit_circuit(nlayer::Int, nrepeat::Int, operator)
    nbit_measure = nbit_virtual = 1
    nbit_used = nbit_measure + nbit_virtual # = 2
    circuit = chain(nbit_used)

    for i=1:nrepeat
        unit = chain(nbit_used)
        #push!(unit, put(nbit_used, 2=>H))
        for j=1:nlayer
            push!(unit, put(nbit_used, 1=>rotor(true, false)))
            push!(unit, put(nbit_used, 2=>H))
            push!(unit, put(nbit_used, 2=>Rz(0.0)))
            push!(unit, control(nbit_used, 1, 2=>shift(0.0)))
            if j == nlayer
                push!(unit, put(nbit_used, 1=>rotor(true, false)))
                push!(unit, put(nbit_used, 2=>H))
                push!(unit, put(nbit_used, 2=>Rz(0.0)))
            end
        end
        #push!(unit, Measure{nbit_used, 1, AbstractBlock}(Z, (1,), 0, false))
        push!(unit, Measure(nbit_used; operator=operator, locs=(1,), resetto=0))
        # GeneralMatrixBlock{Hilbert Dim, Currnet Hilbert Dim, MatrixType??} <: PrimitiveBlock{N}
        # Measure(n::Int; rng=Random.GLOBAL_RNG, operator=ComputationalBasis(), locs=AllLocs(), resetto=nothing, remove=false)
        # resetto: post measured state
        if i==nrepeat # last
            for k=2:nbit_used
                #push!(unit, Measure{nbit_used, 1, AbstractBlock}(Z, (k,), 0, false))
                push!(unit, Measure(nbit_used; operator=operator, locs=(k,), resetto=0))
            end
        end
        push!(circuit, unit)
    end
    dispatch!(circuit, :random)
end

twoqubit_circuit

In [42]:
Measure(2; operator=X, locs=(1,), resetto=0).operator

X

In [43]:
circuit = twoqubit_circuit(1, 3, X)

nqubits: 2
chain
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.49526804127060386)
│  │     └─ rot(Z, 0.7497686124323855)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.7498881752719939)
│  ├─ control(1)
│  │  └─ (2,) shift(0.8972823909685368)
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.19403938015604894)
│  │     └─ rot(Z, 0.9201493874721554)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.613459414492632)
│  └─ Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.8114329090896917)
│  │     └─ rot(Z, 0.4352297242076435)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.9427007016058655)
│  ├─ control(1)
│  │  └─ (2,) shift(0.43682183231133354)
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.5968139764813345)
│  │     └─ rot(Z, 0.7798885549774279)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.4302694004

In [44]:
parameters(circuit)

21-element Array{Float64,1}:
 0.49526804127060386
 0.7497686124323855
 0.7498881752719939
 0.8972823909685368
 0.19403938015604894
 0.9201493874721554
 0.613459414492632
 0.8114329090896917
 0.4352297242076435
 0.9427007016058655
 0.43682183231133354
 0.5968139764813345
 0.7798885549774279
 0.4302694004333465
 0.51296853443736
 0.5807453124733062
 0.15510890348886464
 0.8977222358272794
 0.08944843434553573
 0.019680951609170272
 0.48567093599215716

In [45]:
"""
    gensample(circuit, operator; nbatch=1024) -> Vector of Measure

Generate samples from MPS-inspired circuit. Here, `nbatch` means nshot.
`operator` is the operator to measure.
This function returns a vector of `Measure` gates, results are stored in `m.results`.
"""
function gensample(circuit; nbatch=1024)
    mblocks = collect_blocks(Measure, circuit) # collect all measurement block in circuit
    reg = zero_state(nqubits(circuit); nbatch=nbatch)
    reg |> circuit
    return mblocks
end

gensample

In [46]:
println(collect_blocks(Measure, circuit))
collect_blocks(Measure, circuit)[1].operator

Measure[Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(2,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))]


X

In [47]:
res = gensample(circuit; nbatch=1024)

4-element Array{Measure,1}:
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(2,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))

In [49]:
res[4].results

1024-element Array{Complex{Float64},1}:
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
 -1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
      ⋮
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im

# Model Hamiltonians

Transverse field Ising Model

$H = \sum^{N-1}_{i=1} s^{z}_{i}s^{z}_{i+1} + h\sum^{N}_{i=1}s^{x}_{i}$

In [50]:
"""
for simplicity, we require an AbstractModel contains `size` and `periodic` members.
"""
abstract type AbstractModel{D} end

nspin(model::AbstractModel) = prod(model.size)

nspin (generic function with 1 method)

In [51]:
"""
transverse field ising model, `h` is the strength of transverse field.
"""
struct TFI{D} <:AbstractModel{1}
    size::NTuple{D, Int}
    h::Float64
    periodic::Bool
    TFI(size::Int...; h::Real, periodic::Bool) = new{length(size)}(size, Float64(h), periodic)
end

TFI

In [52]:
function get_bonds(model::AbstractModel{1}) # 1-D
    nbit, = model.size
    [(i, i%nbit+1) for i in 1:(model.periodic ? nbit : nbit-1)]
end

function get_bonds(model::AbstractModel{2}) # 2-D
    m, n = model.size
    cis = LinearIndices(model.size)
    bonds = Tuple{Int, Int, Float64}[]
    for i=1:m, j=1:n
        (i!=m || model.periodic) && push!(bonds, (cis[i,j], cis[i%m+1,j]))
        (j!=n || model.periodic) && push!(bonds, (cis[i,j], cis[i,j%n+1]))
    end
    bonds
end

get_bonds (generic function with 2 methods)

In [53]:
function hamiltonian(model::TFI{1})
    nbit = nspin(model)
    sum(repeat(nbit, Z, (i,j)) for (i,j) in get_bonds(model))*0.25 + # 1/4 * Z_i * Z_j
    sum(put(nbit, i=>X) for i=1:nbit)*0.5model.h # 1/2 * X_i
end

hamiltonian (generic function with 1 method)

In [54]:
tfi_model = TFI(4; h=0.5, periodic=false)

TFI{1}((4,), 0.5, false)

In [55]:
tfi_model.size

(4,)

In [56]:
tfi_h = hamiltonian(tfi_model)

nqubits: 4
+
├─ [scale: 0.25] +
│     ├─ +
│     │  ├─ repeat on (1, 2)
│     │  │  └─ Z
│     │  └─ repeat on (2, 3)
│     │     └─ Z
│     └─ repeat on (3, 4)
│        └─ Z
└─ [scale: 0.25] +
      ├─ +
      │  ├─ +
      │  │  ├─ put on (1)
      │  │  │  └─ X
      │  │  └─ put on (2)
      │  │     └─ X
      │  └─ put on (3)
      │     └─ X
      └─ put on (4)
         └─ X


In [22]:
function ising_energy(circuit, bonds, basis; nbatch=nbatch)
    mblocks = gensample(circuit, basis; nbatch=nbatch) # get measurement blocks and do sampling
    nspin = length(mblocks)
    
    local eng = 0.0
    for (a, b) in bonds
        eng += mean(mblocks[a].results .* mblocks[b].results)
    end
    eng/=4
end

function energy(circuit, model::TFI; nbatch=1024)
    # measuring Z
    eng = ising_energy(circuit, get_bonds(model), Z; nbatch=nbatch)
    # measuring X
    mblocks = gensample(circuit, X; nbatch=nbatch)
    engx = sum(mean.([m.results for m in mblocks]))
    eng + model.h*engx/2
end

energy (generic function with 1 method)

In [23]:
energy(circuit, tfi_model; nbatch=100000)

MethodError: MethodError: no method matching gensample(::ChainBlock{2}, ::ZGate; nbatch=100000)
Closest candidates are:
  gensample(::Any; nbatch) at In[6]:9